In [ ]:
top_dir = '/home/qubic/Calib-TD/'


dataset0 = top_dir+'/2023-03-02/2023-03-02_17.41.14__Fixed-DomeClosed-Amplitude1700'

import numpy as np
from qubicpack.qubicfp import qubicfp
import sys,os
import numpy as np

from qubic import fibtools as ft

import matplotlib.pyplot as plt
from pysimulators import FitsArray

from qubic import fibtools as ft
from qubic.plotters import *
from qubicpack.qubicfp import qubicfp
import qubic.demodulation_lib as dl
import qubic.sb_fitting as sbfit
from qubic.io import write_map

import matplotlib.mlab as mlab
import scipy.ndimage.filters as f

from scipy.signal import argrelextrema, find_peaks, find_peaks_cwt, savgol_filter 

In [ ]:
import bottleneck as bn
from sklearn.cluster import DBSCAN

In [ ]:
a = qubicfp()
a.read_qubicstudio_dataset(dataset0)


In [ ]:
tod = a.tod()
timeaxis = tod[0]
todarray = tod[1]


In [ ]:
np.shape(todarray)

In [ ]:
init = timeaxis[0]
tt = timeaxis - init

First discard the saturated ones 

In [ ]:
def saturation(todarray): 
    
    
    #returns  ok = array of True and False 
    #         bad_idx = idx of the saturated TES in the focalplane 
    #         frac_sat_pertes = fraction of the TOD saturated in the TES
    #         number = number of TES saturated more than 10% in the focalplane
    
    ok = np.ones(256,dtype=bool)
    maxis = np.max(abs(todarray), axis=1)
    upper_satval = np.max(maxis)
    lower_satval = -np.max(maxis)
    badtes = (maxis == np.max(maxis))
    number = badtes.sum() #number of TES saturated in every measurement
    bad_idx = np.array(np.where(badtes==True)) #index of the TES saturated
    bad_idx = np.reshape(bad_idx, (bad_idx.shape[1]))

    frac_sat_pertes = np.zeros((256))
    size = todarray.shape[1]

    for i in range(len(todarray)): 
        mask1 = todarray[i] == upper_satval
        mask2 = todarray[i] == lower_satval
        frac = (np.sum(mask1)+np.sum(mask2))/size
        frac_sat_pertes[i] = frac
    
        if frac_sat_pertes[i] ==0:
            ok[i] = True #good, no saturated
        elif frac_sat_pertes[i] >0.1:
            ok[i] = False #bad, saturated more than 10%
        else: 
            ok[i] = True #good, saturated but less than 10%
            
    return ok, bad_idx, frac_sat_pertes, number

In [ ]:
ok, bad_idx, frac, number = saturation(todarray)

In [ ]:
number

In [ ]:
bad_tod = np.array(np.where(ok==False))
bad_tod = np.reshape(bad_tod, (bad_tod.shape[1]))
good_tod = np.array(np.where(ok==True))
good_tod = np.reshape(good_tod, (good_tod.shape[1]))


These is the list of detectors with saturation

In [ ]:
bad_tod

In [ ]:
len(bad_tod)

In [ ]:
%matplotlib notebook

In [ ]:
for i in range(len(bad_tod)):
    idx_bad = bad_tod[i]
    plt.plot(tt, todarray[idx_bad], label='tod_{}'.format(idx_bad))
plt.xlabel('time')
plt.ylabel('Tod')
plt.title('Tod with more than 10% saturation')
plt.legend()
plt.savefig('Tod_saturation_03-02.png')

List of detectors to analyze 

In [ ]:
good_tod

In [ ]:
%matplotlib notebook

In [ ]:
plt.plot(tt, todarray[29])

In [ ]:
plt.plot(tt, todarray[13])

In [ ]:
plt.plot(tt, todarray[97])

In [ ]:
def jumps_new_2(todarray, size=130, treshold=0.2e6, doplot=False): 
    
    #return xc = beginning of the jump 
    #       xcf = final of the jump 
    #       dif = diference between the final of a jump and the beginning of the next jump 
    #       nc = number of clusters/number of jumps
    
    
    tod_sav = savgol_filter(todarray,window_length=401,polyorder=3, mode='nearest')
    
    def haar(x, size=100):
        out = np.zeros(x.size)
        xf = bn.move_median(x, size)[size:]   
        out[size+size//2:-size+size//2] = xf[:-size] - xf[size:]
        return out
    
    #treshold using that the difference is going to be a value between 0.25e6, 0.5e6, 1e6, 1.5e6, 2e6, 2.5e6
    
    tod_haar_sav = haar(tod_sav, size)
    tod_haar = haar(todarray,size)
    
    #if you don't find any jump then don't make the savgol filter and try with the unfiltered TOD
    if max(abs(tod_haar_sav)) < treshold:
        #if the tood haar also is less than 0.2, probably is not a jump 
        if max(abs(tod_haar)) < 0.2e6:
            number = 0
            print('no jump')
            return number
        else:
            jumps = np.abs(tod_haar) > treshold
            no_sav = True #don't use the savgol filter
    else:
        no_sav = False
        print('use savgol')
        jumps = np.abs(tod_haar_sav) > treshold
    
    idx = np.arange(len(todarray))
    #por el threshold que pones me parece que estas perdiendo valores por eso al indice le resto 50
    idx_jumps = idx[jumps] - 50
    time_jumps = tt[jumps]
        
    
    clust = DBSCAN(eps=size//5, min_samples=1).fit(np.reshape(idx_jumps, (len(idx_jumps),1)))
    nc = np.max(clust.labels_)+1
    xc = np.zeros(nc, dtype=int) 
    xcf = np.zeros(nc, dtype=int)
    for i in range(nc):
        xc[i] = np.min(idx_jumps[clust.labels_ == i])
        xcf[i]= np.max(idx_jumps[clust.labels_ == i])
        
    print(nc)
    delta = xcf - xc #time samples of a jump candidate
    print(delta)
    for i in range(len(delta)):
        if delta[i] < 90:
            print(i)
    del_idx = np.reshape(np.array(np.where(delta<90)),np.array(np.where(delta<90)).shape[1])
    print(len(del_idx))
    xcf = np.delete(xcf, del_idx) #if the amount of time samples is less than 90 probably is not a jump 
    xc = np.delete(xc, del_idx)
    nc -= len(del_idx)
    print(nc)
    
    #if no_sav == False:
    #    for i in range(len(xcf)):
    #        h = abs(tod_sav[xcf[i]+50]-tod_sav[xc[i]-50])
    #        print(h)
        
    if nc!=0:
        dif = np.zeros(nc-1) 
        for j in range(nc):
        #print(j)
            if j < nc-1:
                dif[j] = xc[j+1]-xcf[j]
    else:
        dif = 0.
            
        
    if doplot == True:
        fig, ax = plt.subplots(3, figsize = (10,10))
        if no_sav:
            ax[0].plot(tt, todarray)
            ax2 = ax[0].twinx()
            ax2.plot(tt, tod_haar, color='red', label='haar filter')
            ax[0].set_xlabel('time')
            ax[0].set_ylabel('tod')
            ax2.set_ylabel('haar filter')
            ax2.legend()
        else:
            ax[0].plot(tt, tod_sav)
            ax2 = ax[0].twinx()
            ax2.plot(tt, tod_haar, color='red', label='haar filter')
            ax[0].set_xlabel('time')
            ax[0].set_ylabel('tod')
            ax2.set_ylabel('haar filter')
            ax2.legend()
        
        if no_sav:
            ax[1].plot(tt, todarray)
            ax[1].plot(tt[idx_jumps], todarray[idx_jumps], 'r.', label='time samples jumps')        
            ax[1].set_xlabel('time')
            ax[1].set_ylabel('tod')
            ax[1].legend()
        else:       
            ax[1].plot(tt, tod_sav)
            ax[1].plot(tt[idx_jumps], tod_sav[idx_jumps], 'r.', label='time samples jumps')        
            ax[1].set_xlabel('time')
            ax[1].set_ylabel('tod')
            ax[1].legend()
        
        if no_sav:            
            ax[2].plot(tt, tod_haar, label='haar filter')
            ax[2].plot(tt[idx_jumps], tod_haar[idx_jumps], 'r.')
            ax[2].plot(tt[xc], tod_haar[xc], 'g.')
            ax[2].plot(tt[xcf], tod_haar[xcf], 'g.')
            ax[2].legend()
        else:
            ax[2].plot(tt, tod_haar_sav, label='haar filter')
            ax[2].plot(tt[idx_jumps], tod_haar_sav[idx_jumps], 'r.')
            ax[2].plot(tt[xc], tod_haar_sav[xc], 'g.')
            ax[2].plot(tt[xcf], tod_haar_sav[xcf], 'g.')
            ax[2].legend()
        
    return xc, xcf, dif, nc

In [ ]:
prueba = jumps_new_2(todarray[29],doplot=False)

In [ ]:
xc_29, xcf_29, dif_29, nc_29 = prueba 
tod_29 = todarray[29]

In [ ]:
plt.plot(tt, tod_29)
plt.plot(tt[xc_29], tod_29[xc_29], lw=0,color='red', marker='o', ms=5, label='beginning jump')
plt.plot(tt[xcf_29], tod_29[xcf_29], lw=0,color='green', marker='o', ms=5, label='end jump')
plt.xlabel('time')
plt.ylabel('Tod')
plt.title('TOD 29')
plt.legend()
plt.savefig('tod29_03-02.png')

In [ ]:
prueba_2 = jumps_new_2(todarray[13],doplot=True)

In [ ]:
tod = todarray[good_tod]

Use the function in all the detectors and save the results in a variable called 'result'

In [ ]:
for i in range(len(tod)):
    locals()['result_{}'.format(i)]=jumps_new_2(tod[i],doplot=False)

Split the detectos with jumps detected and no jumps detected

In [ ]:
bad_tod

In [ ]:
indice_jump = np.zeros((len(tod)))
for i in range(len(tod)):        
    resultado = locals()['result_{}'.format(i)]
    if resultado != 0.:
        indice_jump[i] = True 
    else:
        indice_jump[i] = False
        
indice_si = np.array(np.where(indice_jump==True))
indice_no = np.array(np.where(indice_jump==False))

In [ ]:
indice_si = np.reshape(indice_si, (indice_si.shape[1]))

In [ ]:
indice_si

In [ ]:
len(indice_si)

In [ ]:
indice_no = np.reshape(indice_no, (indice_no.shape[1]))

In [ ]:
indice_no

In [ ]:
for i in range(len(indice_si)):
    idx_si = indice_si[i]
    plt.plot(tt, tod[idx_si])
plt.xlabel('time')
plt.ylabel('tod')
plt.title('Tod with Jumps')
plt.savefig('Tod_jumps_03-02.png')

In [ ]:
for i in range(len(indice_no)):
    idx_no = indice_no[i]
    plt.plot(tt, tod[idx_no])
plt.xlabel('time')
plt.ylabel('tod')
plt.title('Tod with No Jumps')
plt.savefig('Tod_nojumps_03-02.png')

In [ ]:
xc_13, xcf_13, dif_13, nc_13 = result_13
tod_13 = tod[13]

In [ ]:
plt.plot(tt, tod[13])
plt.plot(tt[xc_13], tod_13[xc_13], lw=0,color='red', marker='o', ms=5, label='beginning jump')
plt.plot(tt[xcf_13], tod_13[xcf_13], lw=0,color='green', marker='o', ms=5, label='end jump')
plt.xlabel('time')
plt.ylabel('Tod')
#plt.title('TOD 13')
plt.legend()
plt.savefig('tod13_03-02.png')

In [ ]:
xc_19, xcf_19, dif_19, nc_19 = result_19
tod_19 = tod[19]

In [ ]:
plt.plot(tt, tod[19])
plt.plot(tt[xc_19], tod_19[xc_19], lw=0,color='red', marker='o', ms=5, label='beginning jump')
plt.plot(tt[xcf_19], tod_19[xcf_19], lw=0,color='green', marker='o', ms=5, label='end jump')
plt.xlabel('time')
plt.ylabel('Tod')
#plt.title('TOD 19')
plt.legend()
plt.savefig('tod19_03-02.png')

In [ ]:
xc_27, xcf_27, dif_27, nc_27 = result_27
tod_27 = tod[27]

In [ ]:
plt.plot(tt, tod[27])
plt.plot(tt[xc_27], tod_27[xc_27], lw=0,color='red', marker='o', ms=5, label='beginning jump')
plt.plot(tt[xcf_27], tod_27[xcf_27], lw=0,color='green', marker='o', ms=5, label='end jump')
plt.xlabel('time')
plt.ylabel('Tod')
#plt.title('TOD 27')
plt.legend()
plt.savefig('tod27_03-02.png')

In [ ]:
xc_28, xcf_28, dif_28, nc_28 = result_28
tod_28 = tod[28]

In [ ]:
plt.plot(tt, tod[28])
plt.plot(tt[xc_28], tod_28[xc_28], lw=0,color='red', marker='o', ms=5, label='beginning jump')
plt.plot(tt[xcf_28], tod_28[xcf_28], lw=0,color='green', marker='o', ms=5, label='end jump')
plt.xlabel('time')
plt.ylabel('Tod')
#plt.title('TOD 28')
plt.legend()
plt.savefig('tod28_03-02.png')

In [ ]:
xc_30, xcf_30, dif_30, nc_30 = result_30
tod_30 = tod[30]

In [ ]:
plt.plot(tt, tod[30])
plt.plot(tt[xc_30], tod_30[xc_30], lw=0,color='red', marker='o', ms=5, label='beginning jump')
plt.plot(tt[xcf_30], tod_30[xcf_30], lw=0,color='green', marker='o', ms=5, label='end jump')
plt.xlabel('time')
plt.ylabel('Tod')
#plt.title('TOD 30')
plt.legend()
plt.savefig('tod30_03-02.png')

Index in the array of todarray

In [ ]:
index_todarray_jumps = good_tod[indice_si]

In [ ]:
index_todarray_jumps

In [ ]:
index_todarray_nojumps = good_tod[indice_no]

In [ ]:
index_todarray_nojumps

In [ ]:
array_saturated_tod = todarray[bad_tod]
np.save('saturated_tod_0302.npy', array_saturated_tod)
array_jumps_tod = todarray[index_todarray_jumps]
np.save('jumps_tod_0302.npy', array_saturated_tod)
array_nojumps_tod = todarray[index_todarray_nojumps]
np.save('nojumps_tod_0302.npy', array_saturated_tod)
